# Model Registry
https://neptune.ai/blog/ml-experiment-tracking

 <img style="display: block; margin: auto;" src="./images/mlops-experiment-tracking-excalidraw.png" width="1280" height="50">
 
## 1. Motivación
 <img style="display: block; margin: auto;" src="./images/fake-email.png" width="380" height="150">
 
Preguntas que salen a flote con este enfoque:
* ¿Qué ha cambia esta versión del modelo respecto a la anterior?
* ¿Debería actualizar los hiper-parámetros?
* ¿Se necesita algún pre-procesamiento?
* ¿Cuál es el ambiente para poder ejecutarlo? ¿Dependencias? ¿Librerías?
* Si hay un problema en producción, y tengo que volver a la versión anterior, debo buscar el correo.

 <img style="display: block; margin: auto;" src="./images/mlflow-tracking-server-model-registry.png" width="680" height="650">
 
En la clase pasada, hicimos varios experimentos en donde guardamos toda la información respecto a cada uno
+ Modelo
+ Artefactos
+ Metadata de la ejecución
+ Parámetros
+ Etc

Para eso, se hizo uso de un `Tracking server` corriendo en local.

Después de revisar dichos experimentos, se decide que alguno(s) de esos modelos esta(n) listo(s) para un ambiente productivo.

Para ello, se debe registrar dichos modelos en el `Model Registry`.

En ese caso, el ingeniero encargado del deployment podrá revisar cuáles modelos están listos para producción, y así se mejora la comunicación entre la persona encargada de construir el modelo y la persona encargada de deployar el modelo.

El `Model Registry` no realizar el proceso de deployar los modelos, es solamente una herramienta para llevar control de cuáles modelos/versiones están listas para producción.



## 2. Definiciones/Conceptos

https://mlflow.org/docs/latest/model-registry.html#concepts
 
### Modelo

Un modelo de `MLflow` se crea a partir de un experimento o ejecución que se loguea utilizando uno de los métodos `mlflow.<framework>.log_model()` de los flavors de modelo. Una vez logueado, este modelo puede ser registrado en el `Model Registry`.

### Modelo Registrado

Un modelo de `MLflow` puede ser registrado en el `Model Registry`. Un modelo registrado tiene un nombre único, contiene versiones, alias, etiquetas y otros metadatos.

### Versión del Modelo

Cada modelo registrado puede tener una o varias versiones. Cuando se agrega un nuevo modelo al `Model Registry`, se añade como la `versión 1`. Cada nuevo modelo registrado con el mismo nombre de modelo incrementa el número de versión.

### Alias de Modelo

Los alias de modelo te permiten asignar una referencia nombrada y mutable a una versión específica de un modelo registrado. Al asignar un alias a una versión específica del modelo, puedes utilizar ese alias para referenciar dicha versión a través de una URI de modelo o de la API del `Model Registry`. 

Por ejemplo, puedes crear un alias llamado `champion` que apunte a la `versión 1` de un modelo llamado `MyModel`. Luego, puedes referenciar la versión 1 de `MyModel` utilizando la URI `models:/MyModel@champion`.

Los alias son especialmente útiles para la implementación de modelos. Por ejemplo, podrías asignar un alias `champion` a la versión del modelo destinada al tráfico de producción y apuntar a este alias en las cargas de trabajo de producción. Luego, puedes actualizar el modelo que sirve al tráfico de producción reasignando el alias `champion` a una versión diferente del modelo.

### Etiquetas (Tags)

Las etiquetas son pares clave-valor que asocias con modelos registrados y versiones de modelos, lo que te permite etiquetarlos y categorizarlos por función o estado. 

Por ejemplo, podrías aplicar una etiqueta con la clave `task` y el valor `question-answering` (mostrado en la interfaz como `task:question-answering`) a modelos registrados destinados a tareas de preguntas y respuestas. 

A nivel de versión de modelo, podrías etiquetar versiones que están en proceso de validación previa a la implementación con `validation_status:pending` y aquellas aprobadas para su implementación con `validation_status:approved`.


## 3. Model Registry - Hands-On

Si estás ejecutando tu propio servidor de `MLflow`, debes utilizar un backend store con base de datos para acceder al Model Registry a través de la interfaz de usuario (UI) o la API.

Antes de que puedas añadir un modelo al `Model Registry`, debes registrarlo utilizando los métodos `log_model`. 

Una vez que un modelo ha sido registrado, puedes agregar, modificar, actualizar o eliminar el modelo en el Model Registry a través de 

1. La interfaz Gráfica de `MLflow`
    * https://mlflow.org/docs/latest/model-registry.html#ui-workflow
      
2. La API de `MLflow` usando código
    * https://mlflow.org/docs/latest/model-registry.html#ui-workflow 

Antes de ver cómo funciona cada uno, vamos a correr más experimentos:

### 3.1. Continuación del ejemplos `nyc-taxi`

#### Setup

Vamos a retomar el ejemplo de la clase pasada, pero primero vamos a asegurarnos de tener las librerías necesarias instaladas.

```bash
pip install mlflow scikit-learn pandas seaborn hyperopt xgboost fastparquet boto3 pyarrow
```

Ahora vamos a ejecutar el `tracking server` con el siguiente comando y explorar un poco lo que vamos a encontrar
```bash
cd Módulo\ 2/ 
mlflow ui --backend-store-uri sqlite:///mlflow.db
```
   

Creamos una carpeta `data` para descargar los datos necesarios.

In [ ]:
# Create the directory if it doesn't exist
!mkdir -p ./data

# Download files using curl
!curl -o ./data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ./data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

Importar las librerías necesarias y definir función para importar los datos

In [1]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [2]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
df_train = read_dataframe('./data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2024-02.parquet')

Feature Engineering

In [4]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

One Hot Encoding

In [5]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [6]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

Definir los `dataset` como objetos de `mlflow` para poderlos trackear

In [7]:
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

Definir el `tracking URI` y el nombre del experimento

In [8]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name="class-examples-nyc-taxi-experiment")

<Experiment: artifact_location=('file:c:/Users/Rafael/OneDrive - ITESO/2024.3 Otoño/Proyecto de Ciencia de '
 'Datos/ProyCienDatosO2024/Módulo 2/mlruns/1'), creation_time=1726023786909, experiment_id='1', last_update_time=1726023786909, lifecycle_stage='active', name='class-examples-nyc-taxi-experiment', tags={}>

In [9]:
mlflow.sklearn.autolog()

In [10]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

#### Nested Runs

Vamos a ver como podemos encadenar ejecuciones, para ello vamos a definir varios modelos a entrenar:

In [11]:
models = [
    
    {"model": GradientBoostingRegressor,
     "params": {"n_estimators": 100, "learning_rate": 0.3, "max_depth": 25, "random_state": 42},
     },
    
    {"model": ExtraTreesRegressor,
     "params": {"n_estimators": 100, "max_depth": 15, "random_state": 42},
     },
    
    {"model": LinearSVR,
     "params": {"C": 1.0, "epsilon": 0}, 
     },

]

In [13]:
with mlflow.start_run(run_name="Nested Runs"):
    for model in models:
        
        model_class = model["model"]
        model_name = model_class.__name__
        params = model["params"]
        
        with mlflow.start_run(run_name=model_name,nested=True):
            
            ml_model = model_class(**params)
           
            ml_model.fit(X_train, y_train)
    
            y_pred = ml_model.predict(X_val)
            
            rmse = root_mean_squared_error(y_val, y_pred)
            mlflow.log_metric("rmse", rmse)
            
            # !mkdir models
            with open("models/preprocessor.b", "wb") as f_out:
                pickle.dump(dv, f_out)
                
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2024/09/12 20:49:08 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/09/12 20:50:49 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/09/12 20:51:52 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
c:\Users\Rafael\OneDrive - ITESO\2024.3 Otoño\Proyecto de Ciencia de Datos\ProyCienDatosO2024\.venv\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### 3.2 Registrar modelos a través de la UI

https://mlflow.org/docs/latest/model-registry.html#ui-workflow

### 3.3 Registrar modelos a través de código

Hay 3 maneras de registrar un modelo a través de código:

1. `mlflow.<framework>.log_model()`


In [14]:
from sklearn.ensemble import RandomForestRegressor


with mlflow.start_run(run_name="RandomForestRegressor"):
    ml_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        random_state=42
    )
    
    ml_model.fit(X_train, y_train)
    
    mlflow.sklearn.log_model(
        sk_model=model, 
        artifact_path="model",
        registered_model_name="nyc-taxi-model"
    )
    
    y_pred = ml_model.predict(X_val)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    # !mkdir models
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2024/09/12 20:54:33 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/09/12 20:55:37 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/09/12 20:55:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
Created version '5' of model 'nyc-taxi-model'.


En el anterior código, si un modelo con el nombre `nyc-taxi-model` no existe, `MLflow` en automático lo creará y le asignará la `versión 1`. Si ya existe un modelo registrado con ese nombre, el método crea una nueva versión del modelo.

2. La segunda manera es usando el método`mlflow.register_model()`. Después de que todos los `run` terminen y cuando se haya decidido cuál modelo es más adecuado para agregar al `Model Registry`.

In [16]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
Created version '6' of model 'nyc-taxi-model'.


Si un modelo registrado con el nombre no existe "nyc-taxi-model", el método registra un nuevo modelo, crea la `Versión 1` y devuelve un objeto `ModelVersion` de `MLflow`. Si ya existe un modelo registrado con ese nombre, el método crea una nueva versión del modelo y devuelve el objeto de la versión.

3. Finalmente, se puede usar la clase `MlflowClient`.

La clase `MlflowClient` es un cliente para:
* Un `MLflow Tracking Server` que crea y administra `experimentos` y `runs`.
* Un `MLflow Registry Server` que crea y administra modelos registrados y versiones del modelo.

Para instanciar el cliente se usa el siguiente código:


In [17]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

Ahora sí, el tercer camino: 
* Se debe registrar un modelo si aún no existe con el método `create_registered_model()`
* Crear la nueva versión del modelo con el método `create_model_version()`

In [18]:
client.create_registered_model(name="nyc-taxi-model")

MlflowException: Registered Model (name=nyc-taxi-model) already exists. Error: (sqlite3.IntegrityError) UNIQUE constraint failed: registered_models.name
[SQL: INSERT INTO registered_models (name, creation_time, last_updated_time, description) VALUES (?, ?, ?, ?)]
[parameters: ('nyc-taxi-model', 1726196321547, 1726196321547, None)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [19]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = client.create_model_version(
    name="nyc-taxi-model",
    source=run_uri,
    run_id=run_id
)

### 3.4 Asignar aliases y descripciones

In [20]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

# create "champion" alias for version 1 of model "example-model"
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=1
)

# set the "challenger" alias to version 2
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger",
    version=2
)

In [21]:
# get a model version by alias
client.get_model_version_by_alias(
    name="nyc-taxi-model",
    alias="champion"
)

<ModelVersion: aliases=['champion'], creation_timestamp=1726194987841, current_stage='None', description='', last_updated_timestamp=1726194987841, name='nyc-taxi-model', run_id='21357a9c3857418c9888a96bdee7e766', run_link='', source=('file:c:/Users/Rafael/OneDrive - ITESO/2024.3 Otoño/Proyecto de Ciencia de '
 'Datos/ProyCienDatosO2024/Módulo '
 '2/mlruns/1/21357a9c3857418c9888a96bdee7e766/artifacts/model'), status='READY', status_message=None, tags={'status': 'approved'}, user_id=None, version=1>

In [22]:
# delete the alias
client.delete_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger"
)

In [23]:
client.update_model_version(
    name="nyc-taxi-model",
    version=1,
    description="This model version is a scikit-learn random forest containing 100 decision trees",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1726194987841, current_stage='None', description=('This model version is a scikit-learn random forest containing 100 decision '
 'trees'), last_updated_timestamp=1726196458066, name='nyc-taxi-model', run_id='21357a9c3857418c9888a96bdee7e766', run_link='', source=('file:c:/Users/Rafael/OneDrive - ITESO/2024.3 Otoño/Proyecto de Ciencia de '
 'Datos/ProyCienDatosO2024/Módulo '
 '2/mlruns/1/21357a9c3857418c9888a96bdee7e766/artifacts/model'), status='READY', status_message=None, tags={'status': 'approved'}, user_id=None, version=1>

### 3.5 Obteniendo modelos del registro de modelos

1. Obtener el modelo por versión del modelo

In [24]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
model_version = 1

model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.pyfunc.load_model(
    model_uri=model_uri,
)

model.predict(X_val)

array([19.127958, 28.412981,  9.090975, ..., 43.596153, 13.94903 ,
       19.869055], dtype=float32)

2. Obtener el modelo por alias

In [25]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([19.127958, 28.412981,  9.090975, ..., 43.596153, 13.94903 ,
       19.869055], dtype=float32)

### 3.6 Comparación de versiones y selección del nuevo modelo `champion`

En la última sección, recuperaremos los modelos registrados en el `Model Registry` y compararemos su rendimiento en un conjunto de pruebas no visto.
 
La idea es simular el escenario en el que un ingeniero de despliegue tiene que interactuar con el `Model Registry` para decidir si actualizar o no la versión del modelo que está en producción.

Estos son los pasos:

1. Cargar el conjunto de datos de prueba, que corresponde a los datos de los taxis verdes de NYC del mes de marzo de 2024.
2. Descargar el `DictVectorizer` que se ajustó utilizando los datos de entrenamiento y se guardó en `MLflow` como un artefacto, y cargarlo con pickle.
3. Preprocesar el conjunto de pruebas utilizando el `DictVectorizer` para poder alimentar correctamente los regresores.
4. Realizar predicciones en el conjunto de pruebas utilizando las versiones de los modelos que actualmente están en las etapas "challenger" y "champion", y comparar su rendimiento.
5. Con base en los resultados, actualizar la versión del modelo "champion" según corresponda.


In [28]:
# Create the directory if it doesn't exist
!mkdir data

# Download files using curl
!curl -o ./data/green_tripdata_2024-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet

Ya existe el subdirectorio o el archivo data.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1340k  100 1340k    0     0  4546k      0 --:--:-- --:--:-- --:--:-- 4702k


In [29]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [30]:
df = read_dataframe("data/green_tripdata_2024-03.parquet")

In [31]:
run_id = input("Ingrese el run_id")

client.download_artifacts(
    run_id=run_id, 
    path='preprocessor', 
    dst_path='.'
)

'c:\\Users\\Rafael\\OneDrive - ITESO\\2024.3 Otoño\\Proyecto de Ciencia de Datos\\ProyCienDatosO2024\\Módulo 2\\preprocessor'

In [32]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [33]:
X_test = preprocess(df, dv)

In [34]:
target = "duration"
y_test = df[target].values

In [35]:
%time test_model(name="nyc-taxi-model", alias="champion", X_test=X_test, y_test=y_test)

CPU times: total: 1.5 s
Wall time: 660 ms


{'rmse': np.float64(5.194011581181041)}

In [37]:
%time test_model(name=model_name, alias="challenger", X_test=X_test, y_test=y_test)

CPU times: total: 1.3 s
Wall time: 748 ms


{'rmse': np.float64(5.189149261509287)}

In [39]:
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=1
)

: 

## 4. Quiz la siguiente Clase 17 de septiembre de 2024 en hora de clase. Empieza puntual a las 20:10. Será sobre los temas: MLOps, Experiment Tracking y Model Registry; clases 8, 9 y 10 respectivamente.
